In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
!pip install transformers
!pip install datasets
!pip install sacrebleu
!pip insall accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.2 MB/s eta 0:00:0

In [ ]:
from datasets import load_dataset
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split
import datasets
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from datasets import load_metric
import gc
import torch
from transformers import AutoTokenizer
import datasets

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/custom_train_1.csv')
train.head()

,Sentence,English_Translation
0,@someUSER congratulations on you celebrating b...,@some users congratulate you for celebrating B...
1,@LoKarDi_RT uske liye toh bahot kuch karna pad...,"@Lokardi_ rat we should a lot more for that, b..."
2,@slimswamy yehi to hum semjhane ki koshish kar...,"@Slimswami ehi, this is what i'm expecting you..."
3,@DramebaazKudi cake kaha hai ??,@Where is Dramebajakudi where is the cake?
4,@someUSER i'm in hawaii at the moment . home ...,@some user Don't want to come home next friday...


In [ ]:
val = pd.read_csv('/content/drive/MyDrive/st2.csv')
val = val.rename(columns = {'Hinglish' : 'Sentence', 'English' : 'English_Translation'})
val.head()

,Sentence,English_Translation
0,Abe bigg boss show me baki contestants bhi hai...,Now there are other contestants in the Bigg Bo...
1,Filhal toh behanchod zindagi rape kr rae hai.,At present the behanchod life is raping.
2,agar tumhare sath rape ho jaye to tumhare fath...,What will your father do if Rape happens to you?
3,"I hate it, hostel me banti thi. Us din chawal ...",I hate it. was made in hostel. that time i was...
4,"Tumhare to terrorism failane aate Hain, Indian...","You come to spread terrorism, the Indian Army ..."


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/test_st2.csv')
test = test.rename(columns = {'Hinglish' : 'Sentence', 'English' : 'English_Translation'})
test.head()

,Sentence,English_Translation
0,Wah Wah kya baat he par viyapam ghotale ki jaa...,What is that thing? But more than 50 murders h...
1,"Kaun hai eh bhai logo iska bhi rape kardo na, ...","Who is this brother rape her too, this woman i..."
2,Hindu to rape karne ke baad bhi rehta hai ek c...,Hindu remains a criminal even after raped..wha...
3,Ek din ayega ki tumahara hi rape hoga aur jam ...,One day will come and you will be raped and wi...
4,Gujarat BJP ke karyakarta ne rape Kiya he vo K...,Gujarat BJP worker has raped. Show that news o...


In [ ]:
train.shape, val.shape, test.shape

((19386, 2), (500, 2), (1500, 2))

In [ ]:
train_rows = train
val_rows = val
test_rows = test

In [ ]:
def load_val():
    d = []
    for i in range(0, val_rows.shape[0]):
      src = val_rows.iloc[i, :]['Sentence']
      tgt = val_rows.iloc[i, :]['English_Translation']
      print("SRC: ", src)
      print("TRG: ", tgt)
      d.append(
              {
                  "translation": {
                      "hinglish": src.strip(),
                      "english": tgt.strip()
                  }
              }
      )
    print(f'total size of data is {len(d)}')
    tdata = pd.DataFrame(d)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    return dataset

val_set = load_val()

SRC:  Abe bigg boss show me baki contestants bhi hai un se related news deta nhi...sirf sari updates hina se related deta hai...wo bhi hate wali...khabari hai bakri chabar chabar karta rehta hina k bare me
TRG:  Now there are other contestants in the Bigg Boss show and you not giving news related to them. why all updates related to hina? that too hatted... khabri is chatting always about hina.
SRC:  Filhal toh behanchod zindagi rape kr rae hai.
TRG:  At present the behanchod life is raping.
SRC:  agar tumhare sath rape ho jaye to tumhare father kya karenge
TRG:  What will your father do if Rape happens to you?
SRC:  I hate it, hostel me banti thi. Us din chawal me ghee aur namak daal ke khati thi
TRG:  I hate it. was made in hostel. that time i was eating by mixing salt and butter in the rice.
SRC:  Tumhare to terrorism failane aate Hain, Indian army 72 hooron se mila deti unko
TRG:  You come to spread terrorism, the Indian Army would get meet 72 Hurons to them.

SRC:  100 Dino mai Boh

In [ ]:
def load_train():
    d = []
    for i in range(0, train_rows.shape[0]):
      src = train_rows.iloc[i, :]['Sentence']
      tgt = train_rows.iloc[i, :]['English_Translation']
      print("SRC: ", src)
      print("TRG: ", tgt)
      d.append(
              {
                  "translation": {
                      "hinglish": src.strip(),
                      "english": tgt.strip()
                  }
              }
      )
    print(f'total size of data is {len(d)}')
    tdata = pd.DataFrame(d)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    return dataset

train_set = load_train()

In [ ]:
def load_test():
    d = []
    for i in range(0, test_rows.shape[0]):
      src = test_rows.iloc[i, :]['Sentence']
      tgt = test_rows.iloc[i, :]['English_Translation']
      print("SRC: ", src)
      print("TRG: ", tgt)
      d.append(
              {
                  "translation": {
                      "hinglish": src.strip(),
                      "english": tgt.strip()
                  }
              }
      )
    print(f'total size of data is {len(d)}')
    tdata = pd.DataFrame(d)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    return dataset

test_set = load_test()

In [ ]:
test_set

Dataset({
    features: ['id', 'translation'],
    num_rows: 1500
})

In [ ]:
val_set

Dataset({
    features: ['id', 'translation'],
    num_rows: 500
})

In [ ]:
train_set

Dataset({
    features: ['id', 'translation'],
    num_rows: 19386
})

In [ ]:
#!pip install sentencepiece

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[250004, 35378, 4, 903, 1632, 149357, 38, 2], [250004, 3293, 83, 15700, 149357, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "hinglish"
target_lang = "english"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    # no need this line
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train_set = train_set.map(preprocess_function, batched=True)
tokenized_val_set = val_set.map(preprocess_function, batched=True)
tokenized_test_set = test_set.map(preprocess_function, batched=True)

Map:   0%|          | 0/19386 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_set['translation']

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
metric = load_metric("sacrebleu")

<ipython-input-22-b5f524766ed4>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 18.2 MB/s eta 0:00:00


In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

In [ ]:
args = Seq2SeqTrainingArguments(output_dir="./mbart_810/",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=3,
                        per_device_eval_batch_size=3,
                        learning_rate=5e-5,
                        num_train_epochs=3,
                        predict_with_generate=True,
                        logging_dir="/logs",
                        logging_steps=10000,
                        save_steps=10000,
                        report_to="none"
                        )

In [ ]:
trainer = Seq2SeqTrainer(model=model,
                args=args,
                data_collator=data_collator,
                train_dataset=tokenized_train_set,
                eval_dataset=tokenized_val_set,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
                )

tokenized_datasets, tokenized_test_set, trainer2 = accelerator.prepare(
     tokenized_train_set, tokenized_val_set, trainer
      )

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,2.008080,22.898400,31.774000
2,1.521000,2.026594,22.034600,31.132000
3,1.521000,2.320601,21.771300,32.084000


TrainOutput(global_step=19386, training_loss=1.0852791036379346, metrics={'train_runtime': 9716.8666, 'train_samples_per_second': 5.985, 'train_steps_per_second': 1.995, 'total_flos': 5196044930678784.0, 'train_loss': 1.0852791036379346, 'epoch': 3.0})

In [ ]:
# trainer

In [ ]:
#torch.save(model, '/content/mbart_810/model.pt')

In [ ]:
#model_dir = '/content/drive/MyDrive/801_baseline_mbart'
# trainer.save_model(model_dir + 'baseline1')

In [ ]:
# retreive the saved model
#model_retrieved = MBartForConditionalGeneration.from_pretrained(model_dir + 'baseline1',
#                                                        local_files_only=True)

In [ ]:
#model_retrieved

In [ ]:
#trainer = model_retrieved

In [ ]:
predictions = trainer.predict(tokenized_test_set)

In [ ]:
predictions.predictions

array([[     2, 250004, 114590, ...,      1,      1,      1],
       [     2, 250004,  24158, ...,      1,      1,      1],
       [     2, 250004,   4263, ...,      1,      1,      1],
       ...,
       [     2, 250004,   4734, ...,      1,      1,      1],
       [     2, 250004,     52, ...,      1,      1,      1],
       [     2, 250004,  72272, ...,      1,      1,      1]])

In [ ]:
import pickle as pkl

In [ ]:
with open('/content/drive/MyDrive/801_test_predictions_list.pkl', 'wb') as f:
  pkl.dump(predictions.predictions, f)

In [ ]:
predictions.label_ids

array([[250004,  14240,   2685, ...,      1,      1,      1],
       [250004,   1913,  13379, ...,      1,      1,      1],
       [250004,   4865,   1221, ...,      1,      1,      1],
       ...,
       [250004,  15000,   4734, ...,      1,      1,      1],
       [250004,   3293,   1556, ...,      1,      1,      1],
       [250004,  32635,      4, ...,      1,      1,      1]])

In [ ]:
predictions.metrics

{'test_loss': 2.3206005096435547,
 'test_bleu': 21.7713,
 'test_gen_len': 32.084,
 'test_runtime': 223.4204,
 'test_samples_per_second': 2.238,
 'test_steps_per_second': 0.747}

In [ ]:
tokenizer

MBart50TokenizerFast(name_or_path='facebook/mbart-large-50-many-to-many-mmt', vocab_size=250054, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN', 'af_ZA', 'az_AZ', 'bn_IN', 'fa_IR', 'he_IL', 'hr_HR', 'id_ID', 'ka_GE', 'km_KH', 'mk_MK', 'ml_IN', 'mn_MN', 'mr_IN', 'pl_PL', 'ps_AF', 'pt_XX', 'sv_SE', 'sw_KE', 'ta_IN', 'te_IN', 'th_TH', 'tl_XX', 'uk_UA', 'ur_PK', 'xh_ZA', 'gl_ES', 'sl_SI']})

In [ ]:
tokenizer.decode(predictions.predictions[0])

"</s>en_XX Abe big boss doesn't give any news related to other contestants in the show. He gives all the updates related to Minna. He also gives hate related news. He keeps telling everything about Minna.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>"

In [ ]:
for i in range(0, len(predictions.predictions)):
  print("Test data point number ", i)
  print("Source English: ")
  print(tokenized_test_set[i]['translation']['english'])
  print("Source Hinglish: ")
  print(tokenized_test_set[i]['translation']['hinglish'])
  print("Translation: ")
  print(tokenizer.decode(predictions.predictions[i]))
  print()

Test data point number  0
Source English: 
Now there are other contestants in the Bigg Boss show and you not giving news related to them. why all updates related to hina? that too hatted... khabri is chatting always about hina.
Source Hinglish: 
Abe bigg boss show me baki contestants bhi hai un se related news deta nhi...sirf sari updates hina se related deta hai...wo bhi hate wali...khabari hai bakri chabar chabar karta rehta hina k bare me
Translation: 
</s>en_XX Abe big boss doesn't give any news related to other contestants in the show. He gives all the updates related to Minna. He also gives hate related news. He keeps telling everything about Minna.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [ ]:
x_test = []
y_test = []
y_pred = []
for i in range(0, len(predictions.predictions)):
  x_test.append(tokenized_test_set[i]['translation']['english'])
  y_test.append(tokenized_test_set[i]['translation']['hinglish'])
  y_pred.append(tokenizer.decode(predictions.predictions[i]))

In [ ]:
len(x_test), len(y_test), len(y_pred)

(500, 500, 500)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
bleu = []
# reference and predicted sentences
for i in range(0, 1000):
  ref_sent = y_test[i].split()
  pred_sent = y_pred[i].split()
  # calculate BLEU score
  score = sentence_bleu(ref_sent, pred_sent)
  bleu.append(score)

print(bleu)

IndexError: ignored

In [ ]:
import sacrebleu
ref = ['This is a test .']
hyp = ['This is a train .']
bleu = sacrebleu.corpus_bleu(hyp, [ref])
print(bleu.score)

42.72870063962342


In [ ]:
import sacrebleu
bleu = []
# reference and predicted sentences
for i in range(0, len(y_test)):
  ref_sent = [y_test[i]]
  pred_sent = y_pred[i]
  # calculate BLEU score
  score = sacrebleu.corpus_bleu(ref_sent, pred_sent)
  bleu.append(score.score)

print(bleu)

[0.9243411456936733, 4.767707020457095, 0.0, 2.276859592073037, 2.627961710408444, 0.0, 1.9146030690102511, 1.038539619163422, 1.199348129252962, 0.0, 2.1671320168371846, 0.8237054296530683, 1.958224832501124, 1.3338359564519315, 0.0, 0.9243411456936733, 1.5732934811145336, 2.0929373635196202, 1.4763939041893883, 1.5301683686839007, 8.116697886877475, 0.0, 3.377156414337854, 2.908317710573757, 2.0244462660665508, 1.286713095960916, 2.0244462660665508, 0.0, 2.627961710408444, 3.3864985683445354, 1.123099644603982, 1.9822566267103439, 2.8398387225677895, 1.286713095960916, 0.0, 0.0, 1.4091499441909465, 4.767707020457095, 0.0, 1.1599713834969503, 2.1476912089159055, 0.0, 0.0, 2.0540268312306345, 1.6466642419110007, 2.159701133933343, 2.627961710408444, 1.4763939041893883, 1.2414943415352928, 7.809849842300637, 1.526608193588886, 2.0540268312306345, 3.3864985683445354, 0.0, 1.6404472625060698, 1.8160849415439309, 1.7574239156849023, 2.224972978585352, 2.6643211213888947, 4.196114906296549,

In [ ]:
np.sum(bleu)/len(bleu)

1.6464445822630316

In [ ]:
predictions.predictions

array([[     2, 250004, 114590, ...,      1,      1,      1],
       [     2, 250004,  24158, ...,      1,      1,      1],
       [     2, 250004,   4263, ...,      1,      1,      1],
       ...,
       [     2, 250004,   4734, ...,      1,      1,      1],
       [     2, 250004,     52, ...,      1,      1,      1],
       [     2, 250004,  72272, ...,      1,      1,      1]])

In [ ]:
preds = []
for i in range(0, len(predictions.predictions)):
  d = tokenizer.decode(predictions.predictions[i])
  preds.append(d)

In [ ]:
x_test = []
for i in range(0, len(tokenized_test_set)):
  d = tokenized_test_set[i]['translation']['english']
  x_test.append(d)

In [ ]:
y_test = []
for i in range(0, len(tokenized_test_set)):
  d = tokenized_test_set[i]['translation']['hinglish']
  y_test.append(d)

In [ ]:
x_test

['Now there are other contestants in the Bigg Boss show and you not giving news related to them. why all updates related to hina? that too hatted... khabri is chatting always about hina.',
 'At present the behanchod life is raping.',
 'What will your father do if Rape happens to you?',
 'I hate it. was made in hostel. that time i was eating by mixing salt and butter in the rice.',
 'You come to spread terrorism, the Indian Army would get meet 72 Hurons to them.',
 'UP has done much development in hundred days. Whole india is seeing rape, murder, etc. This is the development.',
 'Those who speak ill of Karachi being a victim of terrorism are mourning silently today.',
 "Never watch 140c without Deepika.  chadarmod @iamsrk tiger raped hakle's career in 3.5 days. and you ATG pee drinking? tell you to your star to give a hit movie. chakke  @SRKsDaring",
 'Bad time has come on Nawaz Sharif, which can not be avoided because he got asiq e rasool Mumtaz Qadri hanged, a severe punishment from A

In [ ]:
preds

["</s>en_XX Abe big boss doesn't give any news related to other contestants in the show. He gives all the updates related to Minna. He also gives hate related news. He keeps telling everything about Minna.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>",
 '</s>en_XX Filhal has 

In [ ]:
y_test

['Abe bigg boss show me baki contestants bhi hai un se related news deta nhi...sirf sari updates hina se related deta hai...wo bhi hate wali...khabari hai bakri chabar chabar karta rehta hina k bare me',
 'Filhal toh behanchod zindagi rape kr rae hai.',
 'agar tumhare sath rape ho jaye to tumhare father kya karenge',
 'I hate it, hostel me banti thi. Us din chawal me ghee aur namak daal ke khati thi',
 'Tumhare to terrorism failane aate Hain, Indian army 72 hooron se mila deti unko',
 '100 Dino mai Bohat vikash kiya UP ny pura hindustan dekh raha hy rape murder etc Yahi hy vikas',
 'Karachi ko terrorism ka shikaar honey pe bura bhalla bolnay waaley aaj chup.se sog mana rahey hain.',
 'bina deepika kabhi 140c nahi dekhe chadarmod @iamsrk ne tiger ne hakle ke career ka rape kar diya 3.5 din me aur tu ATG ka moot pee raha? tere star ko bol ek clean hit deke dikhaye chakke @SRKsDaring',
 'Nawaz shareef pay bura waqt aya hwa hy Jo tall nai sakta q k is nay ashiq e rasool MUMTAZ QADRI KO PHA

In [ ]:
x_test

['Now there are other contestants in the Bigg Boss show and you not giving news related to them. why all updates related to hina? that too hatted... khabri is chatting always about hina.',
 'At present the behanchod life is raping.',
 'What will your father do if Rape happens to you?',
 'I hate it. was made in hostel. that time i was eating by mixing salt and butter in the rice.',
 'You come to spread terrorism, the Indian Army would get meet 72 Hurons to them.',
 'UP has done much development in hundred days. Whole india is seeing rape, murder, etc. This is the development.',
 'Those who speak ill of Karachi being a victim of terrorism are mourning silently today.',
 "Never watch 140c without Deepika.  chadarmod @iamsrk tiger raped hakle's career in 3.5 days. and you ATG pee drinking? tell you to your star to give a hit movie. chakke  @SRKsDaring",
 'Bad time has come on Nawaz Sharif, which can not be avoided because he got asiq e rasool Mumtaz Qadri hanged, a severe punishment from A

In [ ]:
preds

["</s>en_XX Abe big boss doesn't give any news related to other contestants in the show. He gives all the updates related to Minna. He also gives hate related news. He keeps telling everything about Minna.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>",
 '</s>en_XX Filhal has 

In [ ]:
import pickle
with open("preds.pkl", "wb") as f:
    # Use the pickle.dump() method to pickle the data to the file
    pickle.dump(preds, f)

In [ ]:
import pickle
with open("y_test.pkl", "wb") as f:
    pickle.dump(y_test, f)

In [ ]:
import pickle
with open("x_test.pkl", "wb") as f:
    pickle.dump(x_test, f)

In [ ]:
preds

["</s>en_XX Abe big boss doesn't give any news related to other contestants in the show. He gives all the updates related to Minna. He also gives hate related news. He keeps telling everything about Minna.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>",
 '</s>en_XX Filhal has 

In [ ]:
stripped_preds = []
for i in preds:
  s = i.replace("<pad>", "")
  s = s.replace("</s>", "")
  s = s.replace("en_XX", "")
  stripped_preds.append(s)

In [ ]:
stripped_preds

[" Abe big boss doesn't give any news related to other contestants in the show. He gives all the updates related to Minna. He also gives hate related news. He keeps telling everything about Minna.",
 ' Filhal has been raped for a long time.',
 ' If you are raped then what will your father do?',
 ' i hate it, i used to stay in hostel. i used to eat ghee and salt in teacup that day.',
 ' you are going to bring terrorism, indian army puts 72 hour period on them',
 ' In the past 100 days, the whole of Uttar pradesh has been underdeveloped, the whole of India is watching rape murder, etc.',
 ' To call Karachi a terrorism prey honey bad place, the tourists are singing soaps today.',
 " without deepika haven't watched 140 c. chadarmod @iamsrk has raped his career in 3.5 days and you are peeing at aTG? tell your star to show a clean hit @SRKsDaring",
 " Nawaz shares the bad news, the one who can't stand tall because of this lover and prophet, Mumtaz is close to Allah.",
 " if you don't hate gu

In [ ]:
stripped_preds[2], x_test[2]

(' If you are raped then what will your father do?',
 'What will your father do if Rape happens to you?')

In [ ]:
import pickle
with open("stripped_preds.pkl", "wb") as f:
    # Use the pickle.dump() method to pickle the data to the file
    pickle.dump(stripped_preds, f)

In [ ]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Example reference translations
references = [stripped_preds[2]]

# Example translations to be evaluated
translations = [x_test[2]]

import sacrebleu

# Calculate the SacreBLEU score
bleu = sacrebleu.corpus_bleu(translations, [references])

print("SacreBLEU score:", bleu.score)

SacreBLEU score: 25.965358893403383


In [ ]:
bleu_list = []
for i in range(len(stripped_preds)):
  references = [stripped_preds[i]]
  translations = [x_test[i]]
  bleu = sacrebleu.corpus_bleu(translations, [references])
  bleu_list.append(bleu.score)

In [ ]:
bleu_list

[12.4613080191179,
 4.8734989388136185,
 25.965358893403383,
 9.38687499151236,
 5.751391809950023,
 11.394659250985377,
 6.047157127164658,
 19.53845407493818,
 1.9508315563589549,
 6.567274736060395,
 58.79423696731584,
 36.93433781793771,
 4.238344437536537,
 47.36678499394922,
 9.22635643555325,
 6.118604182396148,
 15.911783110981515,
 82.24420145878986,
 10.724314736012696,
 4.30799469916529,
 100.00000000000004,
 6.284749190672594,
 52.45537838821977,
 10.992148055152452,
 27.96635629164385,
 23.739904047378587,
 10.419462900243012,
 25.638685577115286,
 45.72313446186435,
 4.016138436407654,
 14.465646898478067,
 15.06443270653063,
 10.127993013562818,
 29.885651564077445,
 9.379601158083608,
 80.91067115702207,
 19.3297739467439,
 16.515821590069027,
 13.432197635122849,
 6.894470562943559,
 6.986768364373987,
 100.00000000000004,
 3.951744928978941,
 8.757447681401308,
 23.56370192487632,
 11.403604443634974,
 22.434531552409887,
 27.72584639330155,
 2.646544824749848,
 9.652

In [ ]:
np.average(bleu_list)

18.528684577752532

In [ ]:
import pickle
with open("blue_list.pkl", "wb") as f:
    # Use the pickle.dump() method to pickle the data to the file
    pickle.dump(bleu_list, f)

In [ ]:
stripped_preds[2]

' If you are raped then what will your father do?'

In [ ]:
x_test[2]

'What will your father do if Rape happens to you?'

In [ ]:
references = [stripped_preds[2]]

In [ ]:
references

['</s>en_XX If you are raped then what will your father do?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']